# Day 3 - Function Calling with LlamaIndex and Azure OpenAI

Welcome to this tutorial on function calling with LlamaIndex and Azure OpenAI! This notebook demonstrates how to use LlamaIndex's function calling capabilities to build a chat interface over a local database.

## Overview

Function calling allows language models to interact with external tools and APIs in a structured way. When you define functions that the model can use, it can:

1. Understand when a function needs to be called
2. Format the arguments correctly
3. Use the function's return value to provide informative responses

In this tutorial, you'll create a system that can:

- Query a local SQLite database
- Parse natural language questions into SQL queries
- Return formatted responses that combine database results with natural language

## Understanding Function Calling

When working with Large Language Models (LLMs) like Azure OpenAI, there are two main approaches to implementing function calling:

### 1. Schema Definition Approach

You can explicitly define the function's interface using a schema that tells the LLM what the function does and how to use it. This is similar to writing an API specification:

```python
function_schema = {
    "name": "list_tables",
    "description": "Retrieve the names of all tables in the database",
    "parameters": {
        "type": "object",
        "properties": {},
        "required": []
    }
}
```

### 2. Auto-inspection Approach

Alternatively, you can let the system automatically inspect your Python functions. For this to work, your functions must have:

- Type hints for parameters and return values
- Detailed docstrings explaining what the function does

```python
def list_tables() -> list[str]:
    """Retrieve the names of all tables in the database.
  
    Returns:
        list[str]: A list of table names in the database
      
    Example:
        >>> list_tables()
        ['products', 'orders', 'customers']
    """
    cursor = db_conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    return [t[0] for t in cursor.fetchall()]
```

The LLM never sees the actual code inside the function - it only sees the type hints and docstring. This is why clear documentation is crucial.



## Setup and Installation

First, install the required packages:

In [ ]:
%pip install llama-index llama-index-core llama-index-llms-azure-openai python-dotenv  ipython-sql

import the necessary packages:

In [1]:
from dotenv import load_dotenv
import os
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.core import (
    VectorStoreIndex, 
    SimpleDirectoryReader,
    SQLDatabase,
    Document,
    Settings
    )
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent

## Configuration

Create a `.env` file with your Azure OpenAI credentials:

```env
AZURE_OPENAI_ENDPOINT="YOUR_AZURE_ENDPOINT"
AZURE_OPENAI_KEY="YOUR_API_KEY"
OPENAI_API_VERSION="Your_API_VERSION" 
```

Set up the Azure OpenAI connection:

In [2]:
load_dotenv()

llm = AzureOpenAI(
    model="gpt-4o",
    engine="gpt-4o",
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    temperature=0.7
)
Settings.llm = llm

## Create a local database

For this minimal example, we'll create a local SQLite database and add some synthetic data so we have something to query.

Load the `sql` IPython extension so we can interact with the database using magic commands (the `%` instructions) to create a new, empty SQLite database.

In [3]:
%load_ext sql
%sql sqlite:///sample.db

Let's create a SQLite database with some sample data:

In [4]:
%%sql
-- Create the 'products' table
CREATE TABLE IF NOT EXISTS products (
  	product_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	product_name VARCHAR(255) NOT NULL,
  	price DECIMAL(10, 2) NOT NULL
  );

-- Create the 'staff' table
CREATE TABLE IF NOT EXISTS staff (
  	staff_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	first_name VARCHAR(255) NOT NULL,
  	last_name VARCHAR(255) NOT NULL
  );

-- Create the 'orders' table
CREATE TABLE IF NOT EXISTS orders (
  	order_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	customer_name VARCHAR(255) NOT NULL,
  	staff_id INTEGER NOT NULL,
  	product_id INTEGER NOT NULL,
  	FOREIGN KEY (staff_id) REFERENCES staff (staff_id),
  	FOREIGN KEY (product_id) REFERENCES products (product_id)
  );

-- Insert data into the 'products' table
INSERT INTO products (product_name, price) VALUES
  	('Laptop', 799.99),
  	('Keyboard', 129.99),
  	('Mouse', 29.99);

-- Insert data into the 'staff' table
INSERT INTO staff (first_name, last_name) VALUES
  	('Alice', 'Smith'),
  	('Bob', 'Johnson'),
  	('Charlie', 'Williams');

-- Insert data into the 'orders' table
INSERT INTO orders (customer_name, staff_id, product_id) VALUES
  	('David Lee', 1, 1),
  	('Emily Chen', 2, 2),
  	('Frank Brown', 1, 3);

 * sqlite:///sample.db
Done.
Done.
Done.
3 rows affected.
3 rows affected.
3 rows affected.


[]

## Define Database Functions

For a database interface, we need three core functions that work together to allow the LLM to explore and query the data:

1. **List Tables**: Shows what tables are available
2. **Describe Table**: Shows the structure of a specific table
3. **Execute Query**: Runs SQL queries

In [5]:
import sqlite3
from typing import List, Tuple, Any

db_file = "sample.db"
db_conn = sqlite3.connect(db_file)

Create a class to handle database operations with type hints and detailed docstrings:

In [6]:
def list_tables() -> List[str]:
    """Retrieve the names of all tables in the database."""
    # Include print logging statements so you can see when functions are being called.
    print(' - DB CALL: list_tables')

    cursor = db_conn.cursor()

    # fetch all the table names.
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    tables = cursor.fetchall()
    
    return [table[0] for table in tables]

list_tables()

 - DB CALL: list_tables


['products', 'sqlite_sequence', 'staff', 'orders']

Once the available tables is known, the next step a database user will need is to understand what columns are available in a given table. Define that too, and test that it works as expected.

In [7]:
def describe_table(table_name: str) -> List[Tuple[str, str]]:
    """
    Look up the table schema.

    Args:
        table_name (str): The name of the table to describe

    Returns: 
        List of columns where each entry is a tuple of (column_name, column_type)
    """
    print(' - DB CALL: describe_table')

    cursor = db_conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name});")
    return [(col[1], col[2]) for col in cursor.fetchall()]

describe_table('products')

 - DB CALL: describe_table


[('product_id', 'INTEGER'),
 ('product_name', 'VARCHAR(255)'),
 ('price', 'DECIMAL(10, 2)')]

Now that the system knows what tables and columns are present, it has enough information to be able to generate and run a `SELECT` query. Now provide that functionality, and test that it works.

In [8]:
def execute_query(sql: str) -> List[List[str]]:
    """
    Execute a SELECT statement and return the results.

    Args:
        sql (str): The SQL query to execute

    Returns:
        List of lists where each inner list contains the row data.
    """
    print(' - DB CALL: execute_query')

    cursor = db_conn.cursor()
    cursor.execute(sql)
    return cursor.fetchall()

execute_query('SELECT * FROM products;')

 - DB CALL: execute_query


[(1, 'Laptop', 799.99), (2, 'Keyboard', 129.99), (3, 'Mouse', 29.99)]

## Implement function calls

Function calling is a powerful capability that allows LLMs to interact with external tools and functions in a structured way. With LlamaIndex, this is implemented through a sophisticated agent framework.

### Core Concepts

1. Tools and Functions
    - In LlamaIndex, functions are wrapped as "Tools" using the `FunctionTool` class.
    - Each tool includes metadata about is purpose and usage.
    - Tools can be Python functions with type hints and docstrings. 
2. ReAct Agent Framework
    - LlamaIndex uses a ReAct (reasoning + action) pattern
    - The agent can:
        - reason about which tools to use
        - plan multiple steps
        - execute tools in sequence
        - combine results into coherent responses

3. Structured Communication
    - Tools communicate with the LLMs through well-defined interfaces.
    - Type hints and docstrings serves as documentation
    - The LLM never sees the actual implementation

Step 1: Tool definition

In [9]:
tools = [
    FunctionTool.from_defaults(
        fn=list_tables,
        # name="list_tables", # this is the name of the function, if not provided, it will be the name of the function
        # description="Get all table names from the database" # this is the description of the function, if not provided, it will be the docstring of the function
    ),
    FunctionTool.from_defaults(
        fn=describe_table,
    ),
    FunctionTool.from_defaults(
        fn=execute_query,
    )
]

Step 2: Agent creation

In [22]:
instruction = """You are a helpful chatbot that can interact with an SQL database for a computer
store. You will take the users questions and turn them into SQL queries using the tools
available. Once you have the information you need, you will answer the user's question using
the data returned. Use list_tables to see what tables are present, describe_table to understand
the schema, and execute_query to issue an SQL SELECT query."""

agent = ReActAgent.from_tools(
    tools = tools,
    verbose=True,
    max_iterations=20,
    context=instruction # this will cover the default ReActChatFormatter
)

Step 3: execution

1. User sends a query to the agent
2. Agent analyzes the query and plans necessary steps
3. Agent selects appropriate tools
4. Tools are executed in sequence
5. Results are combined into a natural language response

In [23]:
# Test with simple queries
response = agent.chat("What is the cheapest product?")
print("Question: What is the cheapest product?")
print(f"Answer: {response}\n")

> Running step 334674f6-3fb3-4e3c-ad4f-762db829bf81. Step input: What is the cheapest product?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: list_tables
Action Input: {}
 - DB CALL: list_tables
Observation: ['products', 'sqlite_sequence', 'staff', 'orders']
> Running step c5bbdd5e-efdd-4e31-b8a7-1e9c40e4bfae. Step input: None
Thought: I need to understand the schema of the 'products' table to find the price column and identify the cheapest product.
Action: describe_table
Action Input: {'table_name': 'products'}
 - DB CALL: describe_table
Observation: [('product_id', 'INTEGER'), ('product_name', 'VARCHAR(255)'), ('price', 'DECIMAL(10, 2)')]
> Running step dc2ac50b-5d42-4921-af1e-7ef49d92c018. Step input: None
Thought: Now that I have the schema of the 'products' table, I can query it to find the cheapest product.
Action: execute_query
Action Input: {'sql': 'SELECT product_name, price FROM products ORDER BY price ASC L

In [24]:
# Test with follow-up questions. This is a more complex query that requires multiple tables and steps
response = agent.chat("Who sold it?")
print("Question: Who sold it?")
print(f"Answer: {response}\n")

> Running step 27ef9e83-7de3-4432-8d83-3b8689525f02. Step input: Who sold it?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: execute_query
Action Input: {'sql': 'SELECT salespersons.name FROM sales JOIN products ON sales.product_id = products.id JOIN salespersons ON sales.salesperson_id = salespersons.id WHERE products.price = (SELECT MIN(price) FROM products)'}
 - DB CALL: execute_query
Observation: Error: no such table: sales
> Running step c3a1b16b-8f04-4fa0-89d9-ed4e071dad0a. Step input: None
Thought: I need to check the available tables in the database to understand the schema and find the relevant tables.
Action: list_tables
Action Input: {}
 - DB CALL: list_tables
Observation: ['products', 'sqlite_sequence', 'staff', 'orders']
> Running step d7593792-e45a-49f0-b614-61917a3a02b7. Step input: None
Thought: There isn't a "sales" table, but there is an "orders" table. I need to check the schema of the "orders" tabl

Inspect the chat history

In [25]:
import textwrap
def print_chat_history(agent):
    """Prints out the chat history from LlamaIndex agent."""
    
    # Get messages from agent's memory
    messages = agent.memory.get()
    
    for msg in messages:
        # Print role (human or assistant)
        print(f"{msg.role.capitalize()}:")
        
        # Print content
        if msg.content:
            print(textwrap.indent(msg.content, "  "))
            
        # If there are additional sources/metadata
        if msg.additional_kwargs:
            print("  Metadata:")
            print(textwrap.indent(str(msg.additional_kwargs), "    "))
            
        print()

print_chat_history(agent)

User:
  What is the cheapest product?

Assistant:
  The cheapest product is the Mouse, which costs $29.99.

User:
  Who sold it?

Assistant:
  The cheapest product was sold by Alice Smith.



## Advanced Features

In this section, we'll explore advanced features that enhance our function calling implementation with LlamaIndex and Azure OpenAI.

### 1. Error Handling
Error handling is crucial for building robust applications. Here's how we implement comprehensive error handling:

In [26]:
from typing import Optional
import logging

class DatabaseError(Exception):
    pass
def safe_execute_query(sql: str) -> Optional[List[Any]]:
    """Safely execute a query with error handling.
    Args:
        sql (str): SQL query to execute
    Returns:
        Optional[List[Any]]: Query results or None if error
    Raises:
        DatabaseError: If query execution fails
    """
    try:
        cursor = db_conn.cursor()
        cursor.execute(sql)
        return cursor.fetchall()
    except sqlite3.Error as e:
        logging.error(f"Database error: {e}")
        raise DatabaseError(f"Failed to execute query: {e}")


Key aspects of error handling:
- Custom exception classes for specific error types
- Detailed error logging for debugging
- Graceful error recovery mechanisms
- Clear error messages for users

### 2. Query Validation
Query validation adds a security layer to prevent unsafe operations:


In [29]:
def validate_query(sql: str) -> bool:
    """Validate that a query is safe to execute.
    
    Args:
        sql (str): SQL query to validate
        
    Returns:
        bool: True if query is safe, False otherwise
    """
    sql = sql.lower()
    # Only allow SELECT statements
    if not sql.strip().startswith('select'):
        return False
    # Block dangerous keywords
    dangerous_keywords = ['drop', 'delete', 'update', 'insert', 'alter', 'truncate']
    return not any(keyword in sql for keyword in dangerous_keywords)

Benefits of query validation:
- Prevents SQL injection attacks
- Ensures read-only operations
- Blocks dangerous database operations
- Validates query structure

In [33]:
advanced_tools = [
    FunctionTool.from_defaults(
        fn=safe_execute_query,
    ),
    FunctionTool.from_defaults(
        fn=validate_query,
    ),
    FunctionTool.from_defaults(
        fn=list_tables,
    ),
    FunctionTool.from_defaults(
        fn=describe_table,
    )
]

# create an agent with the advanced tools
advanced_agent = ReActAgent.from_tools(
    tools=advanced_tools,
    verbose=True,
    max_iterations=20,
    context=instruction
)

# Test the advanced agent
response = advanced_agent.chat("Can you safely check the most expensive product?")
print("\nAgent Response:")
print(response)

> Running step 5f7a9f06-bd33-4172-b04d-7a103f3372b9. Step input: Can you safely check the most expensive product?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: list_tables
Action Input: {}
 - DB CALL: list_tables
Observation: ['products', 'sqlite_sequence', 'staff', 'orders']
> Running step 767d1750-e37f-41c7-881b-19b5a6d20426. Step input: None
Thought: I need to understand the schema of the 'products' table to identify the columns and then safely execute a query to find the most expensive product.
Action: describe_table
Action Input: {'table_name': 'products'}
 - DB CALL: describe_table
Observation: [('product_id', 'INTEGER'), ('product_name', 'VARCHAR(255)'), ('price', 'DECIMAL(10, 2)')]
> Running step 7f0bb54e-3a43-4bab-86dd-4e158d323c1b. Step input: None
Thought: I now have the necessary information about the 'products' table schema. I will construct a query to find the most expensive product.
Action: validate_qu

This code demonstrates:
- Error handling with both valid and invalid queries
- Query validation for different types of SQL statements
- Custom response formatting with Pydantic models
- Integration with the LlamaIndex agent using advanced features